In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import selenium.common.exceptions as sex

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import lxml

import time
from time import sleep
import random
import re

In [2]:
df = pd.read_csv('modoo_v1.csv', index_col=0)
df.head()


,hospital,adress,review_count,link
0,서울리온의원,경기도화성시향남읍,12,https://www.modoodoc.com/hospital/27710/%EC%84...
1,오체안피부과의원,경기도과천시별양동,11,https://www.modoodoc.com/hospital/29604/%EC%98...
2,리즈온의원,서울강동구성내2동,11,https://www.modoodoc.com/hospital/68196/%EB%AE...
3,뮤즈의원,경기도수원시팔달구인계동,13,https://www.modoodoc.com/hospital/67614/%EB%A6...
4,연세베러의원,서울강남구역삼1동,11,https://www.modoodoc.com/hospital/13364/%EC%97...


In [3]:
df.review_count.value_counts()

10    96
13    95
12    93
6     91
5     91
      ..
72     1
64     1
97     1
91     1
70     1
Name: review_count, Length: 80, dtype: int64

In [4]:
work_df = df[df['review_count']<10]
work_df.review_count.unique()
# work_df = work_df[work_df['reviews']>79]
# work_df = work_df[work_df['reviews']<90]
# work_df.reviews.unique()

array([9, 8, 7, 6, 5, 4, 3], dtype=int64)

In [5]:
path = "C:\\chromedriver.exe"
driver = webdriver.Chrome(path)
# driver.implicitly_waits(5)

start_url = 'https://www.modoodoc.com/hospitals/?search_query=%ED%94%BC%EB%B6%80%EA%B3%BC'
driver.get(start_url)

C:\Users\yukir\AppData\Local\Temp/ipykernel_5160/3957373983.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [48]:
root_url = work_df.link.tolist()
print(len(root_url))

urls_list=[]

for j in root_url[:4]:
    root_url = j
    
    for i in range(1,2):
#         print(root_url)
        urls = root_url+('?page='+str(i))
        urls_list.append(urls)
        print(urls)
        
print(len(urls_list))

572
https://www.modoodoc.com/hospital/61536/%EA%B3%A0%EC%9A%B4%EB%82%98%EB%9D%BC%ED%94%BC%EB%B6%80%EA%B3%BC%EC%9D%98%EC%9B%90-%EA%B2%BD%EC%83%81%EB%82%A8%EB%8F%84-%EB%82%B4%EB%8F%99/?page=1
https://www.modoodoc.com/hospital/43597/%EC%9B%B0%ED%94%BC%EB%B6%80%EA%B3%BC%EC%9D%98%EC%9B%90-%EB%8C%80%EA%B5%AC-%EC%88%98%EC%84%B1%EB%8F%991%EA%B0%80/?page=1
https://www.modoodoc.com/hospital/67511/%EC%95%84%EA%B0%80%ED%8C%8C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%82%AC%EC%9A%B0%EB%8F%99/?page=1
https://www.modoodoc.com/hospital/65356/%EC%9B%90%EC%9D%98%EC%9B%90-%EA%B2%BD%EC%83%81%EB%B6%81%EB%8F%84-%EC%96%91%EB%8D%95%EB%8F%99/?page=1
4


In [19]:
reviews = []
stars= []
hoss = []
treatments=[]
addr = []


z=0
j=0

for i in urls_list:
#     j=0
#     print(i)
    driver.get(i)
    driver.implicitly_wait(5)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    
    
    hos = soup.find_all('h1', class_="my-1 hospital-doctor-name-box d-flex align-items-center")
    for i in hos:
        hos = i.text
        hos = hos.strip()
        print('\n>>>>>>>>>>>>>>       {}      <<<<<<<<<<<<<<'.format(hos))
        hoss.append(hos)

    adr = soup.find_all('div', class_="color49 mt-3")
    for i in adr:
        i = i.text
        i = i.strip()
        addr.append(i)
        print(i)
    
    
    review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2") 
    for i in review:
        text = i.text
        #     print(text)
        text = text.replace('\n', '')
        text = text.replace('                        ', '')
        text = text.replace('                    ', '')
        z+=1
        print(str(z)+' //'+hos+'// '+text)
#         print("reviews extracted")
        reviews.append(str(z)+' //'+hos+'// '+text)
        

    star = soup.find_all('span', class_ = "h-100 align-bottom")   
    for i in star:
        i = i.text
        i=i.strip()
        stars.append(i)
        print(i)
    # print("Star/ed")
    
    
    treatment = soup.find_all('div', class_="text-left mt-2 title-box")
    for i in treatment:
        treat = i.text
        treat = treat.replace(' ','')
        treat = treat.replace('\n','')
        treat = treat.split(':')[1]
        treatments.append(treat)
        print(treat)
        
    j+=1

    treatments_ = soup.find_all('div', class_="review-in-detail-text-box-content col-8 pl-5") 

    noyes_list=[]
    noyes_text = []
    for i in treatments_:
        text = i.text
        # print(text)
        text = text.replace('\n', '')
        # print(text)
        text = text.replace('\n', '')
        text = text.replace('                        ', '')
        # print(text)
        # text = text.replace('                    ', '')
        # print(text)
        # text /= text.split('게시        ')[-1]
        # print(text)
        if '받은 치료 :' in text:
            noyes_list.append('있음')
            noyes_text.append('^^있다^^'+text)
            print("있다", text)    #
        elif '받은 치료 : ' not in text:
            noyes_list.append('없음')
            noyes_text.append('없음'+text)
            print("없다", text)
    print('noyes got it')
        
    print('------------     남은 페이지수 {}     ----------------'.format(len(urls_list)-j))
    
    x = random.randint(0,2)
    sleep(x)
            
    print('ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ sleep for {} ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ'.format(x))


>>>>>>>>>>>>>>       고운나라피부과의원      <<<<<<<<<<<<<<
경남 김해시 내외중앙로 57 (내동)
1 //고운나라피부과의원// 두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해주셨어요.제두피에 맞는 샴푸 권하셔서   하나사가지고 왔어요ㆍ샴푸 방법도 자세히 알려주시고참 친절하셨어요,
2 //고운나라피부과의원// 대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 가라 앉았습니다 대상포진 무섭네요 통증이 가라 앉아도 휴유증이 남아서 그 부위가 계속 아프네요  예방 접종 꼭 해야 해요
3 //고운나라피부과의원// 여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을 잘해주셨어요. 지인들에게 추천받아서 갔는데 의사선생님과 진료 후에 상담실로 옮겨가서 치료 프로그램을 고르는 순서로 진행되었어요.먼저 피지 조절하는 아쿠아필 위주로 관리했습니다
4 //고운나라피부과의원// 문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차만별인 치료라 말도 안되게 비싸게 받는 곳 많은데 합리적인 가격에 원하는 부분만 지워주세요 레이저 치료 했습니다 아파요
5 //고운나라피부과의원// 원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! 직원들도 전부 친절해서넘 맘에들었답니다  처방받은약으로 하루만에 ㅠㅠ피부가 돌아와서 아주 만족해요^^~
6 //고운나라피부과의원// 지루성 피부염으로 인해 병원에 다녀왔습니다. 돋보기 같은 것으로 유심히 피부상태를 확인하시고, 이에 따른 연고와 의약품을 처방해주셨습니다. 많은 환자들로 힘드셨을법도 한데, 친절하게 잘 대해주셔서 좋았습니다.
7 //고운나라피부과의원// 발톱무좀때문에 방문했어요 선생님 너무 좋으시더라구요 워낙 소문난 병원이라 방문했는데 역시나 최고였어요!! 세심하게 봐주시고 친절하게 안내도와주셔서 감사합니다 또 약 지으러갈게용^^~
8 //고운나라피부과의원// 과도

In [22]:
len(noyes_text)

13

In [23]:
test_df = pd.DataFrame({'nbyes':noyes_list, 'text':noyes_text})
test_df.head()
# test_df1 = test_df[test_df['text']=='없음'].index
# # test_df1
# df_test = test_df.drop(test_df1)
# work_treatments = df_test['text'].tolist()
# noyes_check_1 = df_test['nbyes'].tolist()
# print(len(work_treatments))
# print(len(noyes_check_1))

,nbyes,text
0,없음,"없음영수증 인증 3월 22일, 2022년 게시 ..."
1,없음,"없음영수증 인증 3월 17일, 2022년 게시 ..."
2,없음,없음
3,있음,"^^있다^^영수증 인증 2월 5일, 2022년 게시 ..."
4,없음,"없음영수증 인증 2월 5일, 2022년 게시 ..."


In [16]:
import pickle
with open ('reviews.pkl','wb') as f:
    pickle.dump(reviews, f)

with open ('stars.pkl','wb') as f:
    pickle.dump(stars, f)

with open ('treatments.pkl','wb') as f:
    pickle.dump(treatments, f)

with open ('addr.pkl','wb') as f:
    pickle.dump(addr, f)


with open ('work_treatments.pkl','wb') as f:
    pickle.dump(work_treatments, f)
    
############################################################################

with open('reviews.pkl', 'rb') as f:
    reviews_check = pickle.load(f)

with open('stars.pkl', 'rb') as f:
    stars_check = pickle.load(f)

with open('treatments.pkl', 'rb') as f:
    treatments_check = pickle.load(f)

with open('addr.pkl', 'rb') as f:
    addr_check = pickle.load(f)

with open('work_treatments.pkl', 'rb') as f:
    work_treatments_check = pickle.load(f)

print(reviews_check[:5])
print(stars_check[:5])
print(work_treatments[:5])
print(addr_check[:5])
print(work_treatments_check[:5])

['1 //고운나라피부과의원// 두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해주셨어요.제두피에 맞는 샴푸 권하셔서   하나사가지고 왔어요ㆍ샴푸 방법도 자세히 알려주시고참 친절하셨어요,', '2 //고운나라피부과의원// 대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 가라 앉았습니다 대상포진 무섭네요 통증이 가라 앉아도 휴유증이 남아서 그 부위가 계속 아프네요  예방 접종 꼭 해야 해요', '3 //고운나라피부과의원// 여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을 잘해주셨어요. 지인들에게 추천받아서 갔는데 의사선생님과 진료 후에 상담실로 옮겨가서 치료 프로그램을 고르는 순서로 진행되었어요.먼저 피지 조절하는 아쿠아필 위주로 관리했습니다', '4 //고운나라피부과의원// 문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차만별인 치료라 말도 안되게 비싸게 받는 곳 많은데 합리적인 가격에 원하는 부분만 지워주세요 레이저 치료 했습니다 아파요', '5 //고운나라피부과의원// 원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! 직원들도 전부 친절해서넘 맘에들었답니다  처방받은약으로 하루만에 ㅠㅠ피부가 돌아와서 아주 만족해요^^~']
['9.3', '10', '10', '10', '9.5']
['없음영수증 인증            3월 22일, 2022년 게시                피부트러블로 방문하였는데 대기환자가 많은편이라 걱정했지만  생각보다 빠르게 진행되었다점심시간끝날쯤이었는데  원무과직원들이 많이 있었지만 응대에 좀 불친절하다고 느껴서 별로 였는데 의사선생님은 굉장히 친절하고 피부상태를 꼼꼼하게 잘살펴주셔서 인상깊었다                    치료 가격정보피부트러블5800 원신고하기                    도움이 되었어요0', '없음영수증 인증            3월 17

In [17]:
print(len(work_treatments_check))

9


In [18]:
print(len(reviews_check))
print(len(stars_check))
print(len(noyes_check_1))
print(len(work_treatments)) ## 받은치료 없음 자료 보고 hospital로 뽑아서 하면 될 듯
print(len(addr_check))  ## work_df 랑 바로 합치면 됨 --> 그리고 바로 저장


36
36
9
9
4


In [86]:
less_10_df = pd.DataFrame({'reviews_check':reviews_check, 'stars':stars_check, 'noyes': noyes_check_1, 'work_treatment':work_treatments})
less_10_df.head()

ValueError: All arrays must be of the same length

In [ ]:
less_10_df_yes = less_10_df[less_10_df['noyes']=='있음']
less_10_df_yes['treats'] = treatments
less_10_df_yes = less_10_df_yes[['reviews_check', 'stars', 'treats']]
less_10_df_yes

C:\Users\yukir\AppData\Local\Temp/ipykernel_8832/3237191214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_10_df_yes['treats'] = treatments


,reviews_check,stars,treats
3,4 //고운나라피부과의원// 문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔...,10,문신제거
6,7 //고운나라피부과의원// 발톱무좀때문에 방문했어요 선생님 너무 좋으시더라구요 워...,10,무좀치료
7,8 //고운나라피부과의원// 과도한 피지분비로 인한 여드름 때문에 방문하였습니다. ...,10,여드름약물치료


In [ ]:
less_10_df_no = less_10_df[less_10_df['noyes']=='없음']
less_10_df_no  = less_10_df_no[['reviews_check', 'stars']]
less_10_df_no

,reviews_check,stars
0,1 //고운나라피부과의원// 두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미...,9.3
1,2 //고운나라피부과의원// 대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약...,10
2,3 //고운나라피부과의원// 여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이...,10
4,5 //고운나라피부과의원// 원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 ...,9.5
5,6 //고운나라피부과의원// 지루성 피부염으로 인해 병원에 다녀왔습니다. 돋보기 같...,10
8,9 //고운나라피부과의원// 생활습관의 개선을 이야기하시는 선생님 말씀이 장기로 보...,9.3


In [35]:
for i in url_list:
    driver.get(i)
    driver.implicitly_wait(5)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    hos = soup.find_all('h1', class_="my-1 hospital-doctor-name-box d-flex align-items-center")

    
    for i in hos:
        hos = i.text
        hos = hos.strip()
        print('\n>>>>>>>>>>>>>>       {}      <<<<<<<<<<<<<<'.format(hos))
        # hoss.append(hos)
    adr = soup.find_all('div', class_="color49 mt-3")
    for i in adr:
        i = i.text
        i = i.strip()
        adress = i
        print(i)

    treatments_ = soup.find_all('div', class_="review-in-detail-text-box-content col-8 pl-5") 

    noyes_list=[]
    noyes_text = []
    hospital = []
    addr = []
    for i in treatments_:
        text = i.text
        # print(text)
        text = text.replace('\n', '')
        # print(text)
        text = text.replace('\n', '')
        text = text.replace('                        ', '')
            # print(text)
            # text = text.replace('                    ', '')
            # print(text)
            # text /= text.split('게시        ')[-1]
            # print(text)
        hospital.append(hos)
        addr.append(adress)

        if '받은 치료 :' in text:
            noyes_list.append('있음')
            noyes_text.append('^^있다^^'+text)
            print("있다", text)    #
        elif '받은 치료 : ' not in text:
            noyes_list.append('없음')
            noyes_text.append('없음'+text)
            print("없다", text)
        else :
            noyes_list.append('몰라')
            noyes_text.append('몰라'+text)
            print("몰리", text)
    print('noyes got it')


    review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2") 
    for i in review:
        text = i.text
            #     print(text)
        text = text.replace('\n', '')
        text = text.replace('                        ', '')
        text = text.replace('                    ', '')
        z+=1
        print(str(z)+' //'+hos+'// '+text)
    #         print("reviews extracted")
        reviews.append(str(z)+' //'+hos+'// '+text)
            

    star = soup.find_all('span', class_ = "h-100 align-bottom")   
    for i in star:
        i = i.text
        i=i.strip()
        stars.append(i)
        print(i)



condi = ['없음']

data = pd.DataFrame({'h':hospital,'adr':addr, 'd': noyes_list, 'f':noyes_text})
data = data[~data['f'].isin(condi)]

reviews = []
stars = []
data['reviews'] = reviews
data['stars'] = stars
data


>>>>>>>>>>>>>>       원의원      <<<<<<<<<<<<<<
경상북도 포항시 북구 천마로 74 (양덕동)
없다 영수증 인증            3월 22일, 2022년 게시                피부트러블로 방문하였는데 대기환자가 많은편이라 걱정했지만  생각보다 빠르게 진행되었다점심시간끝날쯤이었는데  원무과직원들이 많이 있었지만 응대에 좀 불친절하다고 느껴서 별로 였는데 의사선생님은 굉장히 친절하고 피부상태를 꼼꼼하게 잘살펴주셔서 인상깊었다                    치료 가격정보피부트러블5800 원신고하기                    도움이 되었어요0
없다 영수증 인증            3월 17일, 2022년 게시                다리 찰과상으로  몇일 고생하고 있었는데 병원 다녀오고 나서 재생안되고 있던 상처가 피, 진물이 멈추고 재생이 되었어요..!정말 꼭 작은 찰과상이라도 병원 가시면 바로 해결됩니다 ㅠㅠ너무 친절하셨고 소독방법 앞으로 치료 방향도 설명잘해주시고 항생제도 저한테 맞는 항생제로 처방해주셨어요                    치료 가격정보찰과상6200 원신고하기                    도움이 되었어요0
없다 
있다 영수증 인증            2월 5일, 2022년 게시        받은 치료 :            여드름 약물치료시설도 깔끔하고 좋았고 집에서 가까운 거리에 있어서 좋았습니다 의사선생님이 친절하셔서 기분좋게 진료를 마치고 돌아왔었습니다 앞으로도 이 병원을 이용 할 거 같네요 그럼 이상 끝닙니다                    치료 가격정보여드름 약물치료10000 원신고하기                    도움이 되었어요0
없다 영수증 인증            2월 5일, 2022년 게시                의사 선생님이 친절하고 병원이 깨끗했어요  효과 없는 진료는 할 필요 없다고 말씀해주셔서 믿음이 가서 좋았습니다  간호사 선생님들도 친절하셨어

In [45]:
condi = ['없음']

data = pd.DataFrame({'h':hospital,'adr':addr, 'd': noyes_list, 'f':noyes_text})
data = data[~data['f'].isin(condi)]
data.shape

(9, 4)

In [46]:
reviews = []
stars = []

review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2") 
for i in review:
    text = i.text
        #     print(text)
    text = text.replace('\n', '')
    text = text.replace('                        ', '')
    text = text.replace('                    ', '')
    z+=1
    print(str(z)+' //'+hos+'// '+text)
#         print("reviews extracted")
    reviews.append(str(z)+' //'+hos+'// '+text)
        

star = soup.find_all('span', class_ = "h-100 align-bottom")   
for i in star:
    i = i.text
    i=i.strip()
    stars.append(i)
    print(i)

46 //원의원// 피부트러블로 방문하였는데 대기환자가 많은편이라 걱정했지만  생각보다 빠르게 진행되었다점심시간끝날쯤이었는데  원무과직원들이 많이 있었지만 응대에 좀 불친절하다고 느껴서 별로 였는데 의사선생님은 굉장히 친절하고 피부상태를 꼼꼼하게 잘살펴주셔서 인상깊었다
47 //원의원// 다리 찰과상으로  몇일 고생하고 있었는데 병원 다녀오고 나서 재생안되고 있던 상처가 피, 진물이 멈추고 재생이 되었어요..!정말 꼭 작은 찰과상이라도 병원 가시면 바로 해결됩니다 ㅠㅠ너무 친절하셨고 소독방법 앞으로 치료 방향도 설명잘해주시고 항생제도 저한테 맞는 항생제로 처방해주셨어요
48 //원의원// 시설도 깔끔하고 좋았고 집에서 가까운 거리에 있어서 좋았습니다 의사선생님이 친절하셔서 기분좋게 진료를 마치고 돌아왔었습니다 앞으로도 이 병원을 이용 할 거 같네요 그럼 이상 끝닙니다
49 //원의원// 의사 선생님이 친절하고 병원이 깨끗했어요  효과 없는 진료는 할 필요 없다고 말씀해주셔서 믿음이 가서 좋았습니다  간호사 선생님들도 친절하셨어요  사람이 많을 때가서 대기 공간이 부족해서 아쉬웠어요
50 //원의원// 접수처 직원들도 친절하시고 의사선생님도 친절하게 설명해주시고 처방도 해주셨습니다다만 제 증상이 금새 나아지지 않아서 재방문을 해야될거같아서 안타깝습니다어서 깨끗하게 낫고싶습니다
51 //원의원// 의사 선생님이 정말 친절해요!! 여드름 원인과 앞으로 개선방향도 피피티를 보여주시면서 잘 설명해주십니다!대기 장소도 편안한 분위기고, 관리받는 장소도 프라이빗한 느낌이고 다들 친절합니다! 그리고 약처방도 여드름 바로 없어졌어요!
52 //원의원// 팔에 피부발진이 생겨 가려워서 방문을 했는데 제기억에는 PPT자료같은것을 보여주시면서 원인과 증상에 대해 자세히 설명해 주셔서 인상깊었습니다. 전반적으로 무난했습니다. 직접방문해서 경험해보는것을 추천합니다.
53 //원의원// 똑닥으로 미리 예약을 하고 갈수 있어서 대기 시간도 단축할수 있어서 편했어요.  머리에 지루성 두피염

In [47]:
data['reviews'] = reviews
data['stars'] = stars
data

,h,adr,d,f,reviews,stars
0,원의원,경상북도 포항시 북구 천마로 74 (양덕동),없음,"없음영수증 인증 3월 22일, 2022년 게시 ...",46 //원의원// 피부트러블로 방문하였는데 대기환자가 많은편이라 걱정했지만 생각...,9.5
1,원의원,경상북도 포항시 북구 천마로 74 (양덕동),없음,"없음영수증 인증 3월 17일, 2022년 게시 ...",47 //원의원// 다리 찰과상으로 몇일 고생하고 있었는데 병원 다녀오고 나서 재...,10
3,원의원,경상북도 포항시 북구 천마로 74 (양덕동),있음,"^^있다^^영수증 인증 2월 5일, 2022년 게시 ...",48 //원의원// 시설도 깔끔하고 좋았고 집에서 가까운 거리에 있어서 좋았습니다 ...,9.8
4,원의원,경상북도 포항시 북구 천마로 74 (양덕동),없음,"없음영수증 인증 2월 5일, 2022년 게시 ...",49 //원의원// 의사 선생님이 친절하고 병원이 깨끗했어요 효과 없는 진료는 할...,10
6,원의원,경상북도 포항시 북구 천마로 74 (양덕동),있음,"^^있다^^영수증 인증 11월 17일, 2021년 게시 ...",50 //원의원// 접수처 직원들도 친절하시고 의사선생님도 친절하게 설명해주시고 처...,8
7,원의원,경상북도 포항시 북구 천마로 74 (양덕동),있음,"^^있다^^영수증 인증 9월 24일, 2021년 게시 ...",51 //원의원// 의사 선생님이 정말 친절해요!! 여드름 원인과 앞으로 개선방향도...,9.8
9,원의원,경상북도 포항시 북구 천마로 74 (양덕동),없음,"없음영수증 인증 9월 1일, 2021년 게시 ...",52 //원의원// 팔에 피부발진이 생겨 가려워서 방문을 했는데 제기억에는 PPT자...,10
10,원의원,경상북도 포항시 북구 천마로 74 (양덕동),없음,"없음영수증 인증 5월 11일, 2021년 게시 ...",53 //원의원// 똑닥으로 미리 예약을 하고 갈수 있어서 대기 시간도 단축할수 있...,10
12,원의원,경상북도 포항시 북구 천마로 74 (양덕동),있음,"^^있다^^영수증 인증 9월 4일, 2020년 게시 ...",54 //원의원// 한줄평: 편안하고좋은 병원 친절한병원 처방이 적절한병원입니다.아...,9.8
